In [1]:
import numpy as np
import pandas as pd

In [2]:
#načítanie datasetu
data = pd.read_csv("dataset_stroke.csv")

In [3]:
data

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
...,...,...,...,...,...,...,...,...,...,...,...,...
5105,18234,Female,80.0,1,0,Yes,Private,Urban,83.75,NaN,never smoked,0
5106,44873,Female,81.0,0,0,Yes,Self-employed,Urban,125.20,40.0,never smoked,0
5107,19723,Female,35.0,0,0,Yes,Self-employed,Rural,82.99,30.6,never smoked,0
5108,37544,Male,51.0,0,0,Yes,Private,Rural,166.29,25.6,formerly smoked,0


In [4]:
#z datasetu je zrejmé, že posledná osoba ktorá mala mrtvicu má "id" 43424
#ked si danú osobu vypíšeme, zistíme, že prvých 249 záznamov v datasete malo mrtvicu (potom sú už len 0)
last_person_with_stroke = data[data["id"]==43424]
last_person_with_stroke

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
248,43424,Female,78.0,0,0,Yes,Private,Rural,78.81,19.6,Unknown,1


In [5]:
#zahodíme nepotrebné stĺpce, aby sme pracovali len s faktormi, ktoré nás zaujímajú
data = data.drop(data.columns[[0, 6, 7, 8, 9, 10]], axis=1)

In [6]:
data

,gender,age,hypertension,heart_disease,ever_married,stroke
0,Male,67.0,0,1,Yes,1
1,Female,61.0,0,0,Yes,1
2,Male,80.0,0,1,Yes,1
3,Female,49.0,0,0,Yes,1
4,Female,79.0,1,0,Yes,1
...,...,...,...,...,...,...
5105,Female,80.0,1,0,Yes,0
5106,Female,81.0,0,0,Yes,0
5107,Female,35.0,0,0,Yes,0
5108,Male,51.0,0,0,Yes,0


In [7]:
#zoberieme prvých 496 záznamov, polovica mala mrtvicu, druhá nie
data = data.head(496)

In [8]:
data

,gender,age,hypertension,heart_disease,ever_married,stroke
0,Male,67.0,0,1,Yes,1
1,Female,61.0,0,0,Yes,1
2,Male,80.0,0,1,Yes,1
3,Female,49.0,0,0,Yes,1
4,Female,79.0,1,0,Yes,1
...,...,...,...,...,...,...
491,Male,60.0,0,0,Yes,0
492,Male,74.0,0,0,Yes,0
493,Female,23.0,0,0,No,0
494,Female,55.0,0,0,Yes,0
